# Pandas


Pandas (Python Data Analysis Library) is a swiss-army knife module that you'll find at the top of a huge proportion of notebooks. It's popular enough to deserve its own import idiom...

In [ ]:
import pandas as pd

To get a quick idea of the scope of pandas take a look at the autocomplete for pd.<TAB>

In [ ]:
pd.api

The fundamental objects in pandas are the `Series` and the `DataFrame`. Together they encapsulate how you will normally ingest, clean, manipulate and even visualize external data sources.  Most of this module will use methods on these objects so we'll take a quick tour of the concepts they implement, and the idea of an Index.

## Series

The pandas `Series` object is basically a one dimensional indexed array. Schamatically, they look like

| Index | Value |
|-------|-------|
|   0   |  0.12 |
|   1   |  0.24 |
|   2   |  0.36 |
|   3   |  0.48 |


Where there are two columns: an index and a value. Most of the time the index values are distinct (not a firm requirement though!) but they don't have to be integers. Any hashable type will do

In [ ]:
s1 = pd.Series({'one' : 1.0, 'two': 2.0, 'three': 3.0})
s1

One particularly common option is to use a timestamp as the index (don't worry about the syntax here, we'll come back to timeseries later)...

In [ ]:
dti = pd.date_range('2020-06-03', periods=3, freq='H')
pd.Series(['first', 'second', 'third'], index=dti)

The next thing to notice is that the values all have the same type (`dtype: float64` in this case). `pandas` can make `Series` of almost any type as long as all the rows share the same type, but it will try to pick the most efficient implementation (`numpy` `ndarrays` for numeric types). It can fall back to a generic `object` type if all else fails but it really pays (in speed) to keep an eye on the `dtype` and make it a `numpy` type if possible.

In [ ]:
s2 = pd.Series(['one', 2, 'three'])
s2

### Making `Series`

You'll probably find that you are making `DataFrames` more often than `Series`, but lots of operations in `panadas` return Series or want them as an argument so it's good to know how to build them.

Like `numpy` `ndarray`s a sequence will work, it'll get the default ascending integer index

In [ ]:
sf = pd.Series([1.0, 2.0, 3.0, 4.0, 5.0])
sf

In [ ]:
si = pd.Series(range(5))
si

The `__repr__` includes the values we set along with the type of object we have stored (the values). As I mentioned, this is one of the attributes of a Series

In [ ]:
si.dtype

So far, this looks a lot like a numpy array (or even just a list), but we can switch the indexing to suit our needs, by explicitly passing the `index=` arguement

In [ ]:
sm = pd.Series([1., 2., 3., 4., 5.], index=['one', 'two', 'three', 'four', 'five'])
sm

### Indexing and Slicing

In [ ]:
sm['three']

Now the series is starting to look more like a dictionary, in fact, that's a pretty good way to construct series

In [ ]:
sn = pd.Series({'one': 1, 'three': 3, 'two': 2, 'four': 4, 'five': 5})
sn

When you look closely though, a Series has a few tricks that a dictionary doesn't...

In [ ]:
sn['three':'four']

N.B. label based indexes are _inclusive_ of the `stop` value. This is different from most other indexes you'll see in python, but this can cause a little confusion. Basically it boils down to the idea that there isn't always a natural "next" object in a hash (notice that the rows aren't sorted beyond what we specified manually in the index).

If you use numbers to do the slice, you'll get the familiar python behaviour


In [ ]:
sn[1:3]

Series have a `keys()` method, but it returns an index

In [ ]:
sn.keys()

Element by element statements evaluate to Booleans

In [ ]:
sn > 2

and if you remember the material on numpy fancy indexing, this can be very useful

In [ ]:
sn[sn > 2]

There are also some extr indexing methods available to you `.loc`, `.iloc`, `.ix`. Have a look at the documentation for these, they might seem redundant, but actually they're useful in some contexts where ordinary indexing will bite you...

In [ ]:
sa = pd.Series(data=['apple', 'banana', 'orange', 'pineapple'], index=[3, 2, 1, 7])

sa

Suppose we want the value in the 4th row...

In [ ]:
sa[3]

Nope, `3` was interperted as a label. There's an ambiguity because of the type of the index. Fortunately `pandas` gives us a pair of functions to avoid that problem

  * `.iloc[]`: Purely integer-location based indexing.
  * `loc[]` : Purely label based indexing.

In [ ]:
sa.iloc[3]

And `.iloc` will also work with slices

In [ ]:
sa.iloc[:3]

In the opposite situation

In [ ]:
sa.loc[2]

`iloc` does the opposite and tells pandas you want to use the implicit style python notation

In [ ]:
sa.iloc[[2, 1, 3]]

`ix` some combination of these things, but I've never been able to commit it to memory. Fortunately it is being deprecated in favour of `iloc` and `loc` so you don't need to worry 😀

## `DataFrames`

Most of the time you will be using `DataFrames` rather than `Series`, but at a first pass it is OK to think of `DataFrames` as a bunch of `Series` stuck together with a common index. 


| Index | Value1 | Value2 | 
|-------|--------|--------|
|   0   |  0.12  |   'a'  |
|   1   |  0.24  |   'b'  |
|   2   |  0.36  |   'c'  |
|   3   |  0.48  |   'd'  |

The rules discussed above for the index stay the same but notice that we can now have different types in the various columns

In [ ]:
d1 = pd.DataFrame({'floats': sm, 'ints': sn})
d1

### DataFrame Attributes

We've already seen some of the attributes of the DataFrame (column etc.) but there are quite a few available, take a look at `dtypes`, `ndim`, `shape`

In [ ]:
d1.index

In [ ]:
d1.columns

In [ ]:
d1.shape

In [ ]:
d1.dtypes

### Indexing

When indexing a dataframe, the default is to give you the column (you can also use the syntax `d1.ints`)

In [ ]:
d1['ints']


If you are looking for the row, then try `.loc` with the row index value

In [ ]:
d1.loc['one']

As with series, you can give a list instead, but remember to count the parentheses...

In [ ]:
d1.loc[['one','two']]

Or you can use slices

In [ ]:
d1.iloc[:2]

Wherever possible, `pandas` (like `numpy`) will try to return a view on the same data rather than a copy, but because the indexing possibilities in `pandas` are much greater the specific rules are more subtle and we won't get into them now, suffice to say, if you need to break the link between two dataframes you can use the `.copy()` method

In [ ]:
d1a = d1
d1a is d1

In [ ]:
d1b = d1.copy()
d1b is d1

### Changing DataFrames

`DataFrames` are mutable; we can change the values in rows and columns and we can add/remove columns in place. `pandas` will usually try to do this in place, but some modifications (e.g. changing column dtype) require implicit data copies.

In [ ]:
d1.iloc[1] = (3.0, 3)
d1

Add a new column

In [ ]:
d1['ratio'] = d1['ints'] / (2 * d1['floats'])
d1

Sometimes not everything will fit in a single `DataFrame`, we'll see how to concatenate and otherwise join `DataFrames` later on.

## DataFrame Methods

There are *lots* of methods for operating on DataFrames, have a look at the tab completion and explore the documentation for them. In particular, take a look at the help for `describe`, `head` and `tail`. These are great for orienting yourself with a new dataset

In [ ]:
d1.describe()

We won't have time to dive into all of the methods but we'll sample a few, then you should explore the documentation for others. Some of my favourites are

* `min`, `max`
* `mean`, `mode`, `median`
* `max`, `min`, `argmin`, `argmax`, `idxmax`, `idxmin`
* `any`, `all`
* `astype`
* `dropna`
* `shift`
* `sort_index`, `sort_values`

Generally these will return another `DataFrame` with the results you are looking for, but you can also pass the `inplace=True` keyword argument which will modify the `DataFrame` in place and save some memory.

In [ ]:
d1.min()

In [ ]:
(d1 > 3).any()

Think about that 1, `d1 > 3` returns a dataframe of booleans, any checks if there are any true values in each of the columns.

In [ ]:
d1['floats'].astype(int)

In [ ]:
d1.sort_values('floats')

`dropna` comes in VERY handy in combinations with other functions. For example, if your dataframe has a `NaN` in one row of a column, calling `.mean()` on that column won't work, but if you know that value can be safely ignored, you can do `.dropna().mean()`.

### Working with External Data

Data comes in many, many forms from simple csv/json files, real-time APIs, structured binary files and many others. Try `pd.read_<TAB>` to see some of the `pandas` igestion options. `read_csv` is the main workhorse for data sets which will fit on a single machines. It is way more flexible than it's name suggests (S3 buckets, https, compressed files, ...) and many of the arguments to `read_csv` will have equivalents for the other functions, so we'll take a closer look at.
    
We need a CSV to work with. The city of Vancouver has an [open data catalog](https://vancouver.ca/your-government/open-data-catalogue.aspx), which has CSV for some of the datasets. There's a dataset which lists all of the community gardens and food trees maintained by the city. A copy of it is available in this directory called `CommunityGardensAndFoodTrees.csv 

In [ ]:
pd.read_csv?

In [ ]:
gardenDF = pd.read_csv(
    "CommunityGardensAndFoodTrees.csv",
    encoding='latin1',
    delimiter=';'
)
gardenDF.shape

So there are 172 rows, with 19 columns, here are the fist few rows

In [ ]:
gardenDF.head()

So we can tell things like the gardens were created (`YEAR_CREATED`), and where the are (`LATITUDE`, `LONGITUDE`), and who's responsible for them (`STEWARD_OR_MANAGING_ORGANIZATION`). 

Now it is time to clean the data. This is a hugely important step and will generally eat a lot of your time, but it is worth doing right. Having mistakes in your data undermine everything you are trying to do.

First let's look at the index, the default is to index by integer, but we could have picked any column instead. It looks like the first column is unique (`MAPID`) and so let's use that (chosing the index right can make your life much easier when adding data or combining multiple DataFrames). The `inplace=True` argument means modify the existing dataframe rather than returning a modified copy

In [ ]:
gardenDF.set_index('MAPID', inplace=True)

One useful trick when cleaning data is to look at the unique values in a column. You'll often catch coding mistakes or values being used as placeholders this way, e.g.

In [ ]:
gardenDF['YEAR_CREATED'].unique()

`Pre-2010`, `pre-1970` and `pre 2000` are kind of usless (and inconsistent!) so let's toss them

In [ ]:
for badLabel in ['Pre-2010', 'pre-1970', 'pre 2000']:
    gardenDF = gardenDF[gardenDF['YEAR_CREATED'] != badLabel]

gardenDF['YEAR_CREATED'].unique()

There's still a `nan`, and the years are strings (numbers would be better, or even dates). Pandas is pretty smart about dealing with missing data, but that isn't enough there are methods like `dropna()` which will tell pandas to remove them from results or you can remove the problem entries with fancy indexing. For exampe, if we try to convert the `YEAR_CREATED` column to an integer blindly, it will barf on the `nan` (`NaN` is defined for floats but not ints, blame the IEEE)

In [ ]:
gardenDF['YEAR_CREATED'].astype(int)

But we can use the `.notnull` method to figure out where those rows are. This will give us a boolean array which we can use for Fancy Indexing

In [ ]:
gardenDF['YEAR_CREATED'].notnull()

In [ ]:
gardenDF = gardenDF[gardenDF['YEAR_CREATED'].notnull()]
gardenDF

Now we could make `YEAR_CREATED` an integer (or date, see later).

In [ ]:
gardenDF['YEAR_CREATED'] = gardenDF['YEAR_CREATED'].astype(int)

In [ ]:
gardenDF['YEAR_CREATED'].describe()

The column dtype *is* int64, but `describe()` wants floats to work with so it gets converted.

Doing these steps every time for each column can be a lot of work and code, fortunately `read_csv` (and the other ingestion methods) can do most of the work while we're reading in the data. 

  * **delimiter=**: Sometimes a csv is a tsv, tabs are evil
  * **names=**: Pass a list of names to use for the columns
  * **usecols=**: Only slurp up a subset of columns
  * **skiprows=**: Ignore a number of rows at the top of the file
  * **na_values=**: Flag values which the CSV author used to indicate missing data, e.g. -1
  * **encoding=**: ...
  * **converters=**: Do some transformation on the columns before importing them
  * **parse_dates**=: Turn strings into dates
 
Some of these are obvious (e.g. pass in a list of names) but if you look at the documentation they are *really* flexible. Many will accept combinations of columns, functions, dictionaries of functions, etc. For dates you might have years in one column, months in another and days in a third. `parse_date` can combine these into a single `DateTime` object. Other times you might want to transform a column (e.g. convert to int or subtract off a constant), `converters` lets you specify functions to be run on columns during the ingestion.



In [ ]:
import json

geom = gardenDF.loc['FA008']['Geom']
json.loads(geom)['coordinates']

In [ ]:
import json

def geom2lat(geom):
    if geom:
        return json.loads(geom)['coordinates'][0]
    else:
        return np.NaN


gardenDF = pd.read_csv(
    "CommunityGardensAndFoodTrees.csv",
    usecols = [
        'MAPID',
        'YEAR_CREATED',
        'NAME',
        'STEWARD_OR_MANAGING_ORGANIZATION',
        'STREET_NUMBER',
        'STREET_NAME',
        'Geom',
        'Geom'
    ],
    delimiter=';',
    encoding='latin1',
    na_values={
        'YEAR_CREATED': ['Pre-2010', 'pre-1970', 'pre 2000', 'nan']
    },
    index_col='MAPID',
    parse_dates=['YEAR_CREATED'],
    converters={
        'Geom': geom2lat,
    }
)
gardenDF.head()

### Same thing but with JSON/API

[opendata vancouver](https://opendata.vancouver.ca/api/v2/console) publishes the same datasets as an API. The requests module lets you talk to that API and gives you results in JSON to play with. `pandas` can then consume that JSON, e.g.

In [ ]:
import requests

base_url = 'https://opendata.vancouver.ca/api/records/1.0/search/'
headers = {
 'Content-Type': 'application/json; charset=utf-8'
}


params = {
    'dataset' : 'community-gardens-and-food-trees',
    'q'       : '',
    'rows'    : 10,
    'facets'  : ['year_created', 'juristiction', 'geo_local_area']
}


r = requests.get(base_url, params=params)

In [ ]:
r.json().keys()

In [ ]:
r.json()['records'][0]['fields']

So we can do something like

In [ ]:
pd.json_normalize(r.json()['records'])

Which gives us the first 10 rows of our dataframe. Depending on the details of the API (`next_url`, rate limits etc.) the next step would be to make an iterable list of these results and the concatenate them.

## TimeSeries

We've already talked about time and date handling a bit, but I use this a lot, so we can talk about it more! Pandas was created to handle Financial data and do financial modeling. This lineage has given pandas really excellent time handling. The main objects are

  * **Time Stamps**: Specific points in time usually recorded to the second or nanosecond
  * **Time Intervals/Time Deltas**: These types lets you do arithmetic on time objects

We need some dates to play with. There's a convenience function called `to_datetime` which can convert many "human readable" dates to a pd.Timestamp object

In [ ]:
from datetime import datetime

moonwalk = pd.to_datetime('July 20, 1969, 20:17 UTC')
moonwalk

Timestamps have attributes which let you extract days, year, etc. Normally these will be reported as numbers, but the strftime method supports the usual format specifiers (The correspond with the libc specifiers, here's a reference http://strftime.org/)

In [ ]:
print(f"The moon walk took place on a {moonwalk.strftime('%A')}")

In [ ]:
pd.to_datetime(datetime.utcnow(), utc=True)

Let's look at another sample dataset, it contains three colums

  * year
  * month
  * passengers
  
We can combine the year and month to create a date, then we can use the result as the index for a single column dataframe

In [ ]:
flightsDF = pd.read_csv(
    'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/flights.csv',
    parse_dates=[['year','month']],
    index_col='year_month'
)
flightsDF.head()

In [ ]:
flightsDF.dtypes

Now we can index based on date. Let's look at the number of flights in the 1951

In [ ]:
flightsDF['1951':'1952'].plot(kind='bar')

Timestamp objects can also deal with arithmetic

In [ ]:
flightsDF.index[-1] - flightsDF.index[0]

In [ ]:
pd.date_range('2019-06-24 09:00', periods=10, freq='H')

One extremely useful feature with time series is the ability to resample existing time series. For example, we could resample the flight data into year long bins and look at how the mean passenger count increased

In [ ]:
flightsDF.resample('Y').mean().plot(kind='bar')

## Grouping, Joining, Concatenating

You can get pretty far by jamming everything into a single dataframe, but sometimes you might want to do aggregate operations within a dataframe (e.g. group together all of the rows by year and show the mean value of some other column). Alternatively you might want to add new rows to or columns to an existing DataFrame or join dataframes based on shared key.


In [ ]:
carsDF = pd.read_csv(
    'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv'
)
carsDF.head()

### Groupby
Let's group things by number_of_cylinders and see how that affects mpg...

In [ ]:
carsDF['mpg'].mean()

Calling groupby on it's own will give you a `DataFrameGroupBy` object, you have to tell it what you want to do with the groups to actually see some results, this can be convenient if you want to look at different aggregate functions on the same groups.

In [ ]:
carsDFbyCylinders = carsDF.groupby('cylinders')
carsDFbyCylinders

The typical aggregate operations are things like

  * mean()
  * sum()
  * median()
  * min()/max()

In [ ]:
carsDFbyCylinders.mean()

Group the carsDF by model_year and look at the median mpg (don't include the other columns)

In [ ]:
carsDF.groupby('model_year')['mpg'].median()

You can actually do much more with groupby, you can iterate over the groups

In [ ]:
for vroom, group in carsDFbyCylinders:
    print(f"There are {group.shape[0]} cars with {vroom} cylinders")

How are there possibly 4 cars with 3 cylinders?!

We can apply multiple operations at the same time. The `.aggregate()` method can take a list of the operations you want to perform (e.g. ["max", "min"])

In [ ]:
carsDFbyCylinders['mpg'].aggregate(["min","max"])

We can filter based on group, this is a silly example, but group the cars by cylinder, then show me all of the groups with a mean mpg > 15.

In [ ]:
carsDFbyCylinders.filter(lambda x: x['mpg'].mean()>15).head()

The transform method lets you perform a group operation then use the results to update the rows. For example, we could calculate mean values for our groups, then look at how individual cars perform relative to that

In [ ]:
carsDFbyCylinders.transform(lambda x: x - x.mean()).head()

There is also an apply method which is even more general and will let you apply an arbitrary function to the group results.

## Concatenate & Join

There are a handful of functions which handle concatenation. The main workhorse is `pd.concat`, but there are some convenience functions which will let you avoid passing lots of arguments to concat. You can combine either `Series` and `DataFrames` but we'll jump straight to `DataFrames`,

In [ ]:
s1 = pd.DataFrame(
    {
        'upper': ['A', 'B', 'C'], 
        'lower': ['a', 'b', 'c']
    }, 
    columns=['upper', 'lower'], 
    index=[1,2,3]
)

s2 = pd.DataFrame(
    {
        'upper': ['D', 'E', 'F'],
        'lower': ['d', 'e', 'f']
    }, 
    columns=['upper', 'lower'],
    index=[4,5,6]
)

pd.concat([s1,s2])

Notice that we enclosed the thigs we want to join as some sort of iterable (a `list` here).

What if we wanted to add columns rather than rows

In [ ]:
t1 = pd.DataFrame({'upper': ['A', 'B', 'C'], 'lower': ['a', 'b', 'c']}, columns=['upper', 'lower'], index=[1,2,3])
t2 = pd.DataFrame({'greek': ['α', 'β', 'γ']}, index=[1,2,3])

pd.concat([t1, t2], axis='columns')

`pd.concat` will accept duplicate indices, but normally that indicates a problem with the data normalization. `concat` has a `verify_index` argument which can check for these problems and you can specify what you want to do with duplicates manually.

concat will often result in `NaN`s because some columns might not exist in both/all frames.

In [ ]:
d4 = pd.DataFrame({'fruit': ['apple', 'orange'], 'veg': ['brocolli', 'carrot'], 'tree': ['cedar', 'alder']})
d5 = pd.DataFrame({'veg': ['onion', 'potato'], 'fruit': ['banana','grape']})
pd.concat([d4,d5], sort=False, ignore_index=True)

Since the index wasn't important here, I threw it away and just accepted a new one.

In the general case, joining DataFrames can get complex. The concat method can take a `join` keyword to specify a database like join stragegy (inner or outer), but `pd.merge` is a bit more flexible. It implements the usual relations

  * one-to-one (similar to a concat)
  * many-to-one
  * many-to-many
  
For the many-to-one case here is an example

In [ ]:
adf1=pd.DataFrame({
    'class': ['insect', 'spider'], 
    'legs': [6, 8]}
)

adf2=pd.DataFrame({
    'name': ['molly', 'anna', 'stephen', 'mica'], 
    'class': ['insect','insect','spider','insect']}
)                     

print(adf1); print(adf2)

In [ ]:
pd.merge(adf1, adf2)

The _many-to-one_ is many different rows in adf2 being mapped to a single row in adf1 (insects). `pd.merge` also accepts a selection of keyword arguments so you can manually specify which columns to join, patch up name differences etc.